In [20]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import seaborn as sns
from pylab import rcParams

#path = '/kaggle'
#path = '/tmp/working/' # Standard kaggle container data path
path = '/tmp/working/tmp/kaggle-gearbox-analysis/data' # "./"
input_path = path + '/input/gearbox-fault-diagnosis-elaborated-datasets/stdev/'
broken_dataset  = "broken30hz_stdev_100.csv"
healthy_dataset = "healthy30hz_stdev_100.csv"

# Exploring the dataset

In [22]:
# IMPORT DATASETS
healthyDataset = pd.read_csv(input_path + healthy_dataset)
healthyDataset.describe()

,a1,a2,a3,a4,load,failure
count,10154.000000,10154.000000,10154.000000,10154.000000,10154.000000,10154.0
mean,7.064944,4.238840,3.987288,4.343511,45.809533,0.0
std,2.231626,1.326401,1.069945,1.247653,28.315587,0.0
min,1.802930,1.272738,1.022471,1.222780,0.000000,0.0
25%,5.496722,3.337468,3.226538,3.453185,20.000000,0.0
50%,6.856787,4.016475,3.916282,4.275365,50.000000,0.0
75%,8.419728,4.846186,4.689546,5.156541,70.000000,0.0
max,23.909199,10.984053,8.273939,10.609798,90.000000,0.0


In [23]:
brokenDataset = pd.read_csv(input_path + broken_dataset)
brokenDataset.describe()

,a1,a2,a3,a4,load,failure
count,10049.000000,10049.000000,10049.000000,10049.000000,10049.000000,10049.0
mean,4.455177,4.294107,3.706363,4.209203,45.478157,1.0
std,1.202618,0.969298,0.947515,1.205743,29.005078,0.0
min,1.468928,1.341292,0.990336,1.111679,0.000000,1.0
25%,3.595718,3.617281,3.020892,3.358830,20.000000,1.0
50%,4.399122,4.256609,3.646947,4.142668,50.000000,1.0
75%,5.221865,4.949478,4.354815,5.006999,70.000000,1.0
max,10.067158,8.071265,7.425122,10.022236,90.000000,1.0


## Join healthy & broken datasets in a single dataframe

In [ ]:
dataset = pd.concat([healthyDataset, brokenDataset], axis=0)
dataset.describe()

In [ ]:
plt.figure(figsize=(15,28*8))
gs = gridspec.GridSpec(28, 1)
for i, cn in enumerate(dataset[['a1']]):
    ax = plt.subplot(gs[i])
    sns.distplot(dataset[cn][dataset.failure == 1], bins=50)
    sns.distplot(dataset[cn][dataset.failure == 0], bins=50)
    ax.set_xlabel('')
    plt.legend(['broken', 'healthy'])
    ax.set_title('histogram for ' + str(cn))
plt.show()

## Correlation matrix

In [ ]:
rcParams['figure.figsize'] = 15, 8
columns = ['failure','a1', 'a2', 'a3', 'a4', 'load']
sns.heatmap(dataset[columns].corr(),annot=True,cmap='RdYlGn')
fig=plt.gcf()
plt.show()

# Logistic regression

In [ ]:
# Predictor variable (features)
columns = ['a1', 'a2', 'a3', 'a4', 'load']
X = dataset[columns]
# Target variable: Failure (boolean)
#y = dataset.iloc[:,-1]
y = dataset[['failure']]

In [ ]:
# Split the dataset: 80% train, 20% test
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=0)

# Logistic regression classifier
from sklearn.linear_model import LogisticRegression

# Setup the model
logis = LogisticRegression(max_iter=1000)

# Train the model
logis.fit(X_train,y_train)

# Predict Failrure for the test set
y_pred = logis.predict(X_test)
print("Prediction for the test data (first 10 rows):", y_pred[:10])
print(X_test.iloc[:10,:])

## Confusion matrix

In [ ]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_pred,y_test)
cm

In [ ]:
# .score returns the mean accuracy on the given test data and labels
# https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html
print( logis.score(X_test,y_test) , logis.score(X_train,y_train) )

## Classification report

In [ ]:
from sklearn.metrics import classification_report
report = classification_report(y_test,y_pred)
print(report)

## ROC curve

In [ ]:
# For the ROC curve we need not only the predicted class ('Outcome'), but also the scores on what the predictions are based
# * if threshold 0
#  ** predict>0 => Outcome=1
#  ** predict<0 => Outcome=0 
#  - Remember that predictions were calculated with METHOD .predict(X_test)
#  - Now we also have to include METHOD .decision_function(X_test)
y_pred_score = logis.decision_function(X_test)

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report, roc_curve, roc_auc_score, auc

# If using the test set (X_test, y_test)
fpr,tpr,threshold= roc_curve(y_test, y_pred_score)

In [ ]:
# Area Under Curve
lr_auc = auc(fpr, tpr)
print("AUC=", lr_auc)

In [ ]:
# This default size comes from above
#rcParams['figure.figsize'] = 12, 8

plt.figure(figsize = (8,8), dpi=100)

plt.plot(fpr, tpr, color='red', linestyle='-', label = "Logistic Regression (auc  = %0.3f)"%lr_auc)
plt.plot([0,1],[0,1],color='blue',linestyle='--')

plt.xlabel('False Positive Rate (1-specificity)')
plt.ylabel('True Positive Rate (sensitivity)')

plt.legend()

plt.show()